<a href="https://colab.research.google.com/github/Juapatral/webscrapers/blob/main/metrocuadrado_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 6.6MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease

In [ ]:
# -*- coding: utf-8 -*-
print(
"""
#-------------------------------------------------------------------------
# Author: Juan Pablo Trujillo Alviz 
# github: juapatral
# CD: 2021-04-07 
# LUD: 2021-04-07
# Description: webscrapping of metrocuadrado website
# Use: 
#
#
# v1
# Modification:
# Description:
#-------------------------------------------------------------------------
"""
)

# importar librerias
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import json
import time
import glob
import pandas as pd
import datetime as dt

# funciones propias

## extraer detalle informacion
def extraer_detalle_mc(url):

    # obtener driver
    driver.get(url)

    # obtener codigo fuente
    try:
        soup = BeautifulSoup(driver.page_source, "html.parser")
    except:
        return None

    # obtener script pagina
    script = soup.find_all("script", {"type":"application/json"})

    if script == []:
        return None
    
    # find json
    try:
        my_json = json.loads(re.findall("\"realEstate\":(.*})}}}.*$", script[0].text)[0])
    except:
        return None

    # convert to df
    return pd.json_normalize(my_json)

## avanzar pagina
def avance_pagina(driver):

    # obtener vinuclo del siguiente
    element = driver.find_element_by_link_text("Next")

    # crear accion
    actions = ActionChains(driver)

    # moverse hasta el punto
    actions.move_to_element(element).send_keys(Keys.SPACE).perform()

    # dar click
    element.click()

    return None

## particionar lista
def particiones(numb, part=50000):
    return numb//part + (1 if numb%part != 0 else 0)

# parte 1
## configurar opciones del chromedrive
print("---------------Inicia parte 1: configuración del chrome---------------")
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # habilitar en colab
chrome_options.add_argument('--no-sandbox')
#chromedriver = r"D:\jutrujil\Datos\gobierno_informacion_sia\webscraping\driver_v89\chromedriver.exe"
chromedriver = "chromedriver"
driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options)
driver.implicitly_wait(2)

## paginas a buscar
metro_urls = [
    "https://www.metrocuadrado.com/local/arriendo/",
    "https://www.metrocuadrado.com/local/venta/",
    "https://www.metrocuadrado.com/oficinas/arriendo/",
    "https://www.metrocuadrado.com/oficinas/venta/",
    "https://www.metrocuadrado.com/bodega/arriendo/",
    "https://www.metrocuadrado.com/bodega/venta/",
    "https://www.metrocuadrado.com/edificio-de-oficinas/arriendo/",
    "https://www.metrocuadrado.com/edificio-de-oficinas/venta/",
    "https://www.metrocuadrado.com/consultorio/arriendo/",
    "https://www.metrocuadrado.com/consultorio/venta/",
    "https://www.metrocuadrado.com/apartamento/arriendo/",
    "https://www.metrocuadrado.com/apartamento/venta/",
    "https://www.metrocuadrado.com/casa/arriendo/",
    "https://www.metrocuadrado.com/casa/venta/",
    "https://www.metrocuadrado.com/lote/arriendo/",
    "https://www.metrocuadrado.com/lote/venta/",
    "https://www.metrocuadrado.com/finca/arriendo/",
    "https://www.metrocuadrado.com/finca/venta/",
    "https://www.metrocuadrado.com/edificio-de-apartamentos/arriendo/",
    "https://www.metrocuadrado.com/edificio-de-apartamentos/venta/"
]

## alojar las urls de los inmuebles
urls = []

## definir fecha de inicio
now = dt.datetime.now()
fecha = now.year * 10000 + now.month * 100 + now.day

print("Inicia la extracción a las: {}".format(now))

# parte 2
## buscar los inmuebles por cada pagina
print("---------------Inicia parte 2: búsqueda de inmuebles---------------")

urls_df_list = glob.glob("./urls_*.xlsx")

## revisar exista ya un archivo de urls
if urls_df_list == []:
    for metro_url in metro_urls:
        print("Extracción de: " + metro_url)
        
        # cargar la pagina
        driver.get(metro_url)

        # dar click en aceptar cookies
        try:
            driver.find_element_by_link_text("Acepto").click()
        except:
            pass
        
        # convertir a html
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # encontrar la cantidad de inmuebles
        for j in range(3):
            try:
                r = soup.find_all("h1", {"class":"H1-xsrgru-0 jdfXCo d-sm-inline-block breadcrumb-item active"})
                break
            except:
                time.sleep(1)
                j+=1
                if j>=2:
                    driver.refresh()
        
        # definir el numero de paginas a buscar
        try:
            paginas = min([int(re.findall("([0-9]+)", r[0].text)[0])//50, 200])
        except:
            time.sleep(1)
            driver.refresh()
            continue

        # contador de paginas
        contador = 1

        # iniciar extraccion por cada pagina
        for i in range(paginas):
            
            # contador
            contador += 1

            # romper ciclo
            if contador >= paginas*1.2//1:
                break
            
            # convertir a html
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # encontrar todos los inmuebles
            objetos = soup.find_all("a", {"class":"sc-bdVaJa ebNrSm"})
            
            # revisar objetos
            if objetos != []:

                # por cada inmueble
                for objeto in objetos:
                    try:
                        urls.append("https://www.metrocuadrado.com" + objeto["href"])
                    except:
                        pass
            
            for j in range(3):
                try:
                    avance_pagina(driver)
                    break
                except:
                    time.sleep(1)
                    j+=1
                    if j>=2:
                        driver.refresh()
                
            time.sleep(1)
    
    ## crear archivo de urls
    urls_file = "./urls_metrocuadrado-{}.xlsx".format(fecha)

    ## guardar urls
    urls_df = pd.DataFrame(urls)
    with pd.ExcelWriter(urls_file, options={'strings_to_urls': False}) as file:
        urls_df.to_excel(file, index=False)

else:
    urls_df = pd.read_excel(urls_df_list[0])
    try: 
        urls = [*urls_df[0]]
    except:
        urls = [*urls_df["0"]]

## limpiar urls diplicadas
urls = [url for url in urls if url is not None]
urls = [*set(urls)]

# parte 3
## extraer detalle
print("---------------Inicia parte 3: extracción del detalle----------------")
urls = [url for url in urls if url is not None]

## datos
partes = 10000
inicio = 1

## crear particiones
parts = particiones(len(urls), part=partes)

for part in range(inicio, parts):

    ## partir el dataframe
    rango_i, rango_f = partes*part, partes*(part+1)
    rango_f = rango_f if rango_f < len(urls) else len(urls)
    urls_partidas = urls[rango_i:rango_f]
    
    ## crear un dataframe para alojar el detalle
    inm_detalles = pd.DataFrame()
    print("Inicio de extracción de información, parte {}".format(part+1))

    for url_num, url in enumerate(urls_partidas):
        
        # mostrar avance
        texto = (
            "\rProcesado: " + str(url_num+1) + "/" + str(len(urls_partidas)) + 
            " " + str(100*(url_num+1)/len(urls_partidas)) + "%"
        )
        if url_num + 1 == len(urls_partidas):
            texto += "\r"
        print(texto, end="")  

        try:
            inm_detalles = pd.concat([inm_detalles, extraer_detalle_mc(url)])
        except:
            time.sleep(1)
            continue
        ## archivo de detalle
    inms_file = "./metrocuadrado-{}-parte{}.xlsx".format(fecha,part)

    ## exportar a excel
    with pd.ExcelWriter(inms_file, options={'strings_to_urls': False}) as file:
        inm_detalles.to_excel(file, index=False)
    
    ## avisar
    print("Se extrajo la parte {} de {}".format(part+1, parts))

# parte 4
## fin
print("---------------Inicia parte 4: finalización---------------")
driver.close()
print("Extracción finalizada a las {}".format(dt.datetime.now()))


#-------------------------------------------------------------------------
# Author: Juan Pablo Trujillo Alviz 
# github: juapatral
# CD: 2021-04-07 
# LUD: 2021-04-07
# Description: webscrapping of metrocuadrado website
# Use: 
#
#
# v1
# Modification:
# Description:
#-------------------------------------------------------------------------

---------------Inicia parte 1: configuración del chrome---------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: DeprecationWarning: use options instead of chrome_options


Inicia la extracción a las: 2021-04-15 12:54:21.681528
---------------Inicia parte 2: búsqueda de inmuebles---------------
---------------Inicia parte 3: extracción del detalle----------------
Inicio de extracción de información, parte 2
Procesado: 9321/10000 93.21%